In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import resnet50 # preprocess_input, decode_predictions

from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import mobilenet # preprocess_input, decode_predictions

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import inception_v3 # preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from matplotlib.pyplot import imshow
import math
import numpy as np
from glob import glob

### 각 모델 불러오기

In [2]:
base_inception = InceptionV3(weights='imagenet', include_top=True)
model_inception = Model(inputs=base_inception.input, outputs=base_inception.get_layer('avg_pool').output)

In [3]:
base_res = ResNet50(weights='imagenet', include_top=True)
model_res = Model(inputs=base_res.input, outputs=base_res.get_layer('avg_pool').output)

In [4]:
base_mobile = MobileNet(weights='imagenet', include_top=True)
flatten_layer = Flatten()(base_mobile.get_layer('global_average_pooling2d').output)
model_mobile = Model(inputs=base_mobile.input, outputs=flatten_layer)

In [5]:
print(model_inception.input, model_inception.output) # input = 299,299 out : 2048
print(model_res.input, model_res.output)             # input = 224,224 out : 2048
print(model_mobile.input, model_mobile.output)       # input = 224,224 out : 1024

KerasTensor(type_spec=TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") KerasTensor(type_spec=TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None), name='avg_pool/Mean:0', description="created by layer 'avg_pool'")
KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'") KerasTensor(type_spec=TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None), name='avg_pool/Mean:0', description="created by layer 'avg_pool'")
KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'") KerasTensor(type_spec=TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None), name='flatten/Reshape:0', description="created by layer 'flatten'")


### 데이터 준비

In [2]:
def load_images_and_labels(files, label, size):
    images = [image.load_img(p, target_size=(size, size)) for p in files]  # warping
    labels = [label] * len(files)
    print(files)
    return images, labels

In [3]:
# 파일 경로에 지정, 이미지 로드
man_file= glob('../data/imgs/data_mw/man/*jpg')
wom_file= glob('../data/imgs/data_mw/woman/*jpg')

In [4]:
man_file_299_x, man_file_299_y = load_images_and_labels(man_file, 0, 299)
wom_file_299_x, wom_file_299_y = load_images_and_labels(wom_file, 1, 299)
man_file_224_x, man_file_224_y = load_images_and_labels(man_file, 0, 224)
wom_file_224_x, wom_file_224_y = load_images_and_labels(wom_file, 1, 224)

['../data/imgs/data_mw/man\\ma_1.jpg', '../data/imgs/data_mw/man\\ma_10.jpg', '../data/imgs/data_mw/man\\ma_100.jpg', '../data/imgs/data_mw/man\\ma_101.jpg', '../data/imgs/data_mw/man\\ma_102.jpg', '../data/imgs/data_mw/man\\ma_103.jpg', '../data/imgs/data_mw/man\\ma_104.jpg', '../data/imgs/data_mw/man\\ma_105.jpg', '../data/imgs/data_mw/man\\ma_106.jpg', '../data/imgs/data_mw/man\\ma_107.jpg', '../data/imgs/data_mw/man\\ma_108.jpg', '../data/imgs/data_mw/man\\ma_109.jpg', '../data/imgs/data_mw/man\\ma_11.jpg', '../data/imgs/data_mw/man\\ma_110.jpg', '../data/imgs/data_mw/man\\ma_111.jpg', '../data/imgs/data_mw/man\\ma_113.jpg', '../data/imgs/data_mw/man\\ma_114.jpg', '../data/imgs/data_mw/man\\ma_115.jpg', '../data/imgs/data_mw/man\\ma_116.jpg', '../data/imgs/data_mw/man\\ma_117.jpg', '../data/imgs/data_mw/man\\ma_118.jpg', '../data/imgs/data_mw/man\\ma_119.jpg', '../data/imgs/data_mw/man\\ma_12.jpg', '../data/imgs/data_mw/man\\ma_121.jpg', '../data/imgs/data_mw/man\\ma_122.jpg', '../

In [5]:
images_299 = man_file_299_x + wom_file_299_x
labels = man_file_299_y + wom_file_299_y
vector_299 = np.asarray([image.img_to_array(img) for img in images_299])

images_224 = man_file_224_x + wom_file_224_x
vector_224 = np.asarray([image.img_to_array(img) for img in images_224])

In [6]:
print(vector_299.shape)
print(vector_224.shape)

(337, 299, 299, 3)
(337, 224, 224, 3)


### 머신러닝을 위한 각 모델별 예측

In [11]:
input_for_inception_v3 = (inception_v3.preprocess_input(vector_299))
input_for_resnet50 = (resnet50.preprocess_input(vector_224))
input_for_mobilenet = (mobilenet.preprocess_input(vector_224))

In [12]:
model_inception_ml_input = model_inception.predict(input_for_inception_v3)
model_res_ml_input = model_res.predict(input_for_resnet50)
model_mobile_ml_input = model_mobile.predict(input_for_mobilenet)

11/11 [==============================] - 1s 20ms/step


In [13]:
pretrained_inputs = [model_inception_ml_input, model_res_ml_input, model_mobile_ml_input]

### 각 머신러닝 모델 돌려보고 결과 확인

In [14]:
import warnings

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
import xgboost as xgb

from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
models = [LinearSVC() ,MLPClassifier() ,GaussianProcessClassifier() ,LogisticRegression(), SVC(probability=True), GaussianNB(), DecisionTreeClassifier(),
          RandomForestClassifier(), KNeighborsClassifier(), GradientBoostingClassifier(),
          AdaBoostClassifier(), ExtraTreesClassifier(), HistGradientBoostingClassifier(),
          LinearDiscriminantAnalysis(), QuadraticDiscriminantAnalysis(), BaggingClassifier(),
          RidgeClassifier(), Perceptron(), MLPClassifier(), GaussianProcessClassifier(),
          ComplementNB(), BernoulliNB(), NuSVC(), xgb.XGBClassifier()]

best_model_score = 0

with open('1.transfer_with_ML_result.csv', 'a') as f:  # 'a' 모드는 append 모드입니다.
            f.write('pretrain_name,ml_name,정확도,f1_score')

for name, X in zip(['inception','res50','mobile'],pretrained_inputs):
    x_train, x_test, y_train, y_test = train_test_split(X, labels,
                                                        test_size=0.2, random_state=42)
    for model in models:
        model.fit(x_train,y_train)
        model_score = model.score(x_test,y_test)
        model_name = str(model)[:25].ljust(25)
        f1 = f1_score(y_test, model.predict(x_test))
        result_save = f'{name[:7].ljust(7)},{model_name},{model_score:0.4f},{f1:0.4f}\n'
        print(result_save)

        with open('1.transfer_with_ML_result.csv', 'a') as f:  # 'a' 모드는 append 모드입니다.
            f.write(result_save)

incepti,LinearSVC()              ,0.8088,0.7636

incepti,MLPClassifier()          ,0.8382,0.8000

incepti,GaussianProcessClassifier,0.7500,0.6792

incepti,LogisticRegression()     ,0.8382,0.8070

incepti,SVC(probability=True)    ,0.8529,0.8214

incepti,GaussianNB()             ,0.8088,0.7719

incepti,DecisionTreeClassifier() ,0.7794,0.7273

incepti,RandomForestClassifier() ,0.8382,0.8000

incepti,KNeighborsClassifier()   ,0.7500,0.6531

incepti,GradientBoostingClassifie,0.8382,0.8000

incepti,AdaBoostClassifier()     ,0.8382,0.7925

incepti,ExtraTreesClassifier()   ,0.8971,0.8727

incepti,HistGradientBoostingClass,0.8824,0.8519

incepti,LinearDiscriminantAnalysi,0.8676,0.8525

incepti,QuadraticDiscriminantAnal,0.5735,0.4912

incepti,BaggingClassifier()      ,0.8382,0.7843

incepti,RidgeClassifier()        ,0.7941,0.7407

incepti,Perceptron()             ,0.8824,0.8519

incepti,MLPClassifier()          ,0.8529,0.8148

incepti,GaussianProcessClassifier,0.7500,0.6792

incepti,ComplementNB

### 전이 학습 with 머신러닝

In [2]:
import pandas as pd

metrics_pandas = pd.read_csv('1_2_transfer_with_ML_2_result.csv')

In [3]:
metrics_pandas

,pretrain_name,ml_name,정확도,f1_score
0,incepti,LinearSVC(),0.8088,0.7636
1,incepti,MLPClassifier(),0.8382,0.8000
2,incepti,GaussianProcessClassifier,0.7500,0.6792
3,incepti,LogisticRegression(),0.8382,0.8070
4,incepti,SVC(probability=True),0.8529,0.8214
...,...,...,...,...
67,mobile,GaussianProcessClassifier,0.7647,0.7037
68,mobile,ComplementNB(),0.8676,0.8302
69,mobile,BernoulliNB(),0.8529,0.8276
70,mobile,NuSVC(),0.8824,0.8462


In [42]:
print('최고의 정확도의 인덱스',metrics_pandas.iloc[metrics_pandas['정확도'].argmax()].name)
print('최고의 f1 스코어 인덱스',metrics_pandas.iloc[metrics_pandas['f1_score'].argmax()].name)
print('\n최고 성능\n',metrics_pandas.iloc[metrics_pandas['f1_score'].argmax()])

### 모바일 넷의 로지스틱 회귀 우승! ###

최고의 정확도의 인덱스 51
최고의 f1 스코어 인덱스 51

최고 성능
 pretrain_name                      mobile 
ml_name          LogisticRegression()     
정확도                                 0.9118
f1_score                            0.8929
Name: 51, dtype: object


# 모바일 넷의 로지스틱 회귀 우승! #